In [ ]:
%pip install tensorflow
%pip install transformers
%pip install datasets
%pip install pillow
%pip install numpy
%pip install matplotlib
%pip install datasets

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if IS_COLAB:
    import os
    os.environ["TF_USE_LEGACY_KERAS"] = "1"
    import tf_keras

import tensorflow as tf

tf.random.set_seed(42)

In [ ]:
from datasets import load_dataset

DATASET_SIZE = 23298
dataset = load_dataset("microsoft/cats_vs_dogs")

def to_rgb(example):
  if example["image"].mode != "RGB":
    example["image"] = example["image"].convert("RGB")
  return example

dataset = load_dataset("microsoft/cats_vs_dogs")
dataset = dataset.filter(lambda example: example["labels"] == 0).map(to_rgb)

tf_dataset = dataset["train"].to_tf_dataset(
  shuffle=True,
  batch_size=None
)

def dict_to_tuple(data):
    image = tf.cast(data['image'], tf.float32)
    image = (image / 127.5) - 1.0  # map [0,255] → [-1,1]
    return image, tf.cast(data['labels'], tf.float32)

def resize(image, label):
    image = tf.image.resize(image, [48, 64])
    return image, label

tf_dataset = tf_dataset.map(dict_to_tuple).map(resize)

In [ ]:
import matplotlib.pyplot as plt

def show_image(numpy_image):
  # extra code — plot images during training
  plt.imshow(numpy_image)
  plt.axis("off")
  plt.show()

for i, l in tf_dataset.skip(20).take(5):
  show_image(i.numpy())

In [ ]:
from tensorflow.keras import layers

batch_size = 128

final_dataset = tf_dataset.take(10000).batch(batch_size).prefetch(1)

tf.random.set_seed(42)

codings_size = 256

generator = tf.keras.Sequential([
    layers.Dense(6 * 8 * 256),
    layers.Reshape([6, 8, 256]),
    layers.BatchNormalization(),
    layers.Conv2DTranspose(128, kernel_size=7, strides=2,
                                    padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv2DTranspose(64, kernel_size=5, strides=2,
                                    padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv2DTranspose(3, kernel_size=5, strides=2,
                                    padding="same", activation="tanh"),
])

discriminator = tf.keras.Sequential([
    layers.Input(shape=(48, 64, 3)),
    layers.Conv2D(64, kernel_size=5, strides=2, padding="same",
                        activation=layers.LeakyReLU(0.2)),
    layers.Dropout(0.4),
    layers.Conv2D(128, kernel_size=5, strides=2, padding="same",
               activation=layers.LeakyReLU(0.2)),
    layers.Dropout(0.4),
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

gan = tf.keras.Sequential([generator, discriminator])

# DEV-NOTE
# rmsprop seems to give better results to around 50 epochs
# i think it is worth a while to test out adam optimizer at 0.002 learning rate

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
import time

epochs = 50

# DEV-NOTE
# this should be later updated into better learning loop
# based on gradient tape

# this training loop is heavly inspired by the example in:
# https://github.com/ageron/handson-ml3
def train_gan(gan, dataset, batch_size, codings_size, n_epochs):
    generator, discriminator = gan.layers
    print("training started.")
    for epoch in range(n_epochs):
        start = time.time()
        for X_batch, _ in dataset:
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)

            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            discriminator.trainable = False

            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            gan.train_on_batch(noise, y2)


        print(f"Epoch {epoch + 1}/{n_epochs} - time: {time.time()-start}")  # extra code
        # extra code — plot images during training
        plt.imshow(generated_images.numpy()[0])
        plt.axis("off")
        plt.show()

train_gan(gan, final_dataset, batch_size, codings_size, epochs)